# Chuẩn bị môi trường

In [1]:
%pip install -q paddlepaddle-gpu "paddleocr>=2.0.1"

import paddle
paddle.utils.run_check()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


- Một số lỗi trong quá trình train :
  - Phiên bản CUDA không tương thích nếu train bằng GPU
    ```
    cudaErrorNoKernelImageForDevice: no kernel image is available for execution on the device.
    ```
  - Chưa cài đặt cuDNN

In [17]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Thu Aug  8 05:30:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0              29W /  7

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

In [ ]:
## Download dataset ctw1500
# !wget "https://paddleocr.bj.bcebos.com/dataset/ctw1500.zip"
# !unzip ctw1500.zip

In [4]:
import os
import shutil
import random
import yaml

# Cây thư mục

In [37]:
from rich import print
from rich.tree import Tree
import os

def generate_tree(directory, max_files=5, max_depth=None):
    tree = Tree(f"[bold purple]:open_file_folder: {os.path.basename(directory)}")
    add_to_tree(tree, directory, max_files, max_depth, current_depth=0)
    return tree

def add_to_tree(tree, directory, max_files, max_depth, current_depth):
    if max_depth is not None and current_depth >= max_depth:
        return

    items = sorted(os.listdir(directory))
    dirs = [item for item in items if os.path.isdir(os.path.join(directory, item))]
    files = [item for item in items if os.path.isfile(os.path.join(directory, item))]

    for dir_name in dirs:
        path = os.path.join(directory, dir_name)
        branch = tree.add(f"[bold purple]:open_file_folder: {dir_name}")
        add_to_tree(branch, path, max_files, max_depth, current_depth + 1)

    for i, file_name in enumerate(files):
        if i < max_files:
            tree.add(f"[bold yellow]:page_facing_up: {file_name}")
        elif i == max_files:
            tree.add("[bold yellow]...")
            break

print(generate_tree("/content/drive/MyDrive/Paddle_OCR/dataset", max_files=5, max_depth=None))

📂 dataset
├── 📂 .ipynb_checkpoints
├── 📂 images
│   ├── 📄 000001.png
│   ├── 📄 000002.png
│   ├── 📄 000003.png
│   ├── 📄 000004.png
│   ├── 📄 000005.png
│   └── ...
├── 📂 original_images
│   ├── 📄 2aaa.png
│   ├── 📄 2aag.png
│   ├── 📄 2aam.png
│   ├── 📄 2aba.png
│   ├── 📄 2abe.png
│   └── ...
├── 📄 en_dict.txt
├── 📄 train_list.txt
├── 📄 v3_config.yml
├── 📄 v4_config.yml
└── 📄 val_list.txt

In [5]:
import os

def generate_directory_tree(startpath, prefix='', max_files=5):
    output = []
    if prefix == '':
        output.append('└── ' + os.path.basename(startpath))
        prefix = '    '

    contents = sorted(os.listdir(startpath))
    folders = [item for item in contents if os.path.isdir(os.path.join(startpath, item))]
    files = [item for item in contents if os.path.isfile(os.path.join(startpath, item))]

    for i, folder in enumerate(folders):
        if i == len(folders) - 1 and len(files) == 0:
            output.append(prefix + '└── ' + folder)
            output.extend(generate_directory_tree(os.path.join(startpath, folder), prefix + '    ', max_files))
        else:
            output.append(prefix + '├── ' + folder)
            output.extend(generate_directory_tree(os.path.join(startpath, folder), prefix + '│   ', max_files))

    for i, file in enumerate(files[:max_files]):
        if i == len(files) - 1 or (i == max_files - 1 and len(files) > max_files):
            output.append(prefix + '└── ' + file)
        else:
            output.append(prefix + '├── ' + file)

    if len(files) > max_files:
        output.append(prefix + '└── ...')

    return output

# Sử dụng hàm
startpath = '/content/drive/MyDrive/Paddle_OCR/dataset'
tree = generate_directory_tree(startpath)
print('\n'.join(tree))

└── dataset
    ├── .ipynb_checkpoints
    ├── images
    │   ├── 000001.png
    │   ├── 000002.png
    │   ├── 000003.png
    │   ├── 000004.png
    │   └── 000005.png
    │   └── ...
    ├── original_images
    │   ├── 2aaa.png
    │   ├── 2aag.png
    │   ├── 2aam.png
    │   ├── 2aba.png
    │   └── 2abe.png
    │   └── ...
    ├── en_dict.txt
    ├── train_list.txt
    ├── v3_config.yml
    ├── v4_config.yml
    └── val_list.txt


# Chuẩn bị dữ liệu

In [6]:
# Xóa folder
folder_path = ""

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")
else:
    print(f"Folder '{folder_path}' does not exist.")

Folder '' does not exist.


In [47]:
# Kiểm tra folder
folder_path = '/content/drive/MyDrive/Paddle_OCR/dataset/original_images'
if os.path.isfile(folder_path) == False:
  file_list = os.listdir(folder_path)
  print(f"{os.path.basename(folder_path)} : {len(file_list)} files")

original_images : 50 files

In [ ]:
def split_and_label(source_folder, save_data_dir, train_ratio=0.8):
    # Đường dẫn đến các thư mục
    image_folder = os.path.join(save_data_dir, "images")
    train_label_file = os.path.join(save_data_dir, "train_list.txt")
    val_label_file = os.path.join(save_data_dir, "val_list.txt")

    # Tạo các thư mục nếu chưa tồn tại
    for folder in [image_folder]:
        os.makedirs(folder, exist_ok=True)

    # Lấy danh sách tất cả các file ảnh
    image_files = [f for f in os.listdir(source_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    # random.shuffle(image_files) # Xáo trộn vị trí

    # Tính số lượng ảnh cho tập train
    num_train = int(len(image_files) * train_ratio)

    # Chia và di chuyển ảnh
    for i, file_name in enumerate(image_files):
        old_path = os.path.join(source_folder, file_name)
        file_extension = os.path.splitext(file_name)[1] # định dạng file (.jpg)
        label_name = os.path.splitext(file_name)[0]     # tên file (2apd, ...)
        label_name = label_name.split('_')[0]           # Loại bỏ dấu '_' nếu có trong file_name
        new_filename = f"{i+1:06d}{file_extension}"     # đặt tên mới cho file ảnh : 000001.jpg, 000002.jpg, ...

        if i < num_train:
            label_list_file = train_label_file
        else:
            label_list_file = val_label_file

        new_path = os.path.join(image_folder, new_filename)

        # Di chuyển và đổi tên file ảnh
        shutil.copy2(old_path, new_path) # ghi đè nếu file đã tồn tại
        print(f"Copied from {old_path} to {new_path} successful")

        # Cấu trúc nhãn : relative path + 1 tab + nhãn ( tên file )
        # ( lưu ý : nếu trong tên có dấu '_' thì lấy phần phía trước dấu '_' )
        with open(label_list_file, "a", encoding="utf-8") as f:
            relative_path = os.path.relpath(new_path, start=save_data_dir)
            f.write(f"{relative_path}\t{label_name}\n")

    print(f"Đã chia {num_train} ảnh vào tập train và {len(image_files) - num_train} ảnh vào tập val.")

# Sử dụng hàm
source_folder = "/content/drive/MyDrive/Paddle_OCR/dataset/original_images" # Folder ảnh
save_data_dir = "/content/drive/MyDrive/Paddle_OCR/dataset" # Folder muốn lưu dataset
split_and_label(source_folder, save_data_dir)

Copied from /content/drive/MyDrive/Paddle_OCR/dataset/original_images/2apd.png to /content/drive/MyDrive/Paddle_OCR/dataset/images/000001.png successful
Copied from /content/drive/MyDrive/Paddle_OCR/dataset/original_images/2agd.png to /content/drive/MyDrive/Paddle_OCR/dataset/images/000002.png successful
Copied from /content/drive/MyDrive/Paddle_OCR/dataset/original_images/2abe.png to /content/drive/MyDrive/Paddle_OCR/dataset/images/000003.png successful
Copied from /content/drive/MyDrive/Paddle_OCR/dataset/original_images/2adf.png to /content/drive/MyDrive/Paddle_OCR/dataset/images/000004.png successful
Copied from /content/drive/MyDrive/Paddle_OCR/dataset/original_images/2anx.png to /content/drive/MyDrive/Paddle_OCR/dataset/images/000005.png successful
Copied from /content/drive/MyDrive/Paddle_OCR/dataset/original_images/2ang.png to /content/drive/MyDrive/Paddle_OCR/dataset/images/000006.png successful
Copied from /content/drive/MyDrive/Paddle_OCR/dataset/original_images/2aka.png to 

# Chỉnh sửa cấu hình

- Sửa cấu hình theo thứ tự chuẩn

In [4]:
import yaml
from collections import OrderedDict

def ordered_load(stream, Loader=yaml.Loader, object_pairs_hook=OrderedDict):
    class OrderedLoader(Loader):
        pass
    def construct_mapping(loader, node):
        loader.flatten_mapping(node)
        return object_pairs_hook(loader.construct_pairs(node))
    OrderedLoader.add_constructor(
        yaml.resolver.BaseResolver.DEFAULT_MAPPING_TAG,
        construct_mapping)
    return yaml.load(stream, OrderedLoader)

def ordered_dump(data, stream=None, Dumper=yaml.Dumper, **kwds):
    class OrderedDumper(Dumper):
        pass
    def _dict_representer(dumper, data):
        return dumper.represent_dict(data.items())
    OrderedDumper.add_representer(OrderedDict, _dict_representer)
    return yaml.dump(data, stream, OrderedDumper, **kwds)

In [5]:
# File config của paddleOCR cho english recognition (en_rec)
# /PaddleOCR/configs/rec/
config_path = './dataset/v3_config.yml'

- Đọc và chuyển đổi dữ liệu theo đúng thứ tự
  - Ngoài ra có thể sử dụng cách đọc không theo thứ tự
  - Không ảnh hưởng đến kết quả ( Train có thể đứng trước Global )
  ```python
    with open(config_path, 'r') as file:
      config = yaml.safe_load(file)
    ...
    modified_config_str = yaml.dump(config)
  ```

In [6]:
with open(config_path, 'r') as file:
  config = ordered_load(file, yaml.SafeLoader)

train_data_dir = './dataset/'
val_data_dir = './dataset/'
train_label_dir = './dataset/train_list.txt'
val_label_dir = './dataset/val_list.txt'

In [20]:
config['Global']['use_gpu'] = True
config['Global']['epoch_num'] = 10
config['Global']['save_model_dir'] = './output/rec_ppocr_v3'
config['Global']['eval_batch_step'] = [0, 200]
config['Global']['pretrained_model'] = './models/en_PP-OCRv3_rec_train/best_accuracy'
config['Global']['character_dict_path'] = './dataset/en_dict.txt'
config['Optimizer']['lr']['learning_rate'] = 0.001
config['Train']['dataset']['data_dir'] = train_data_dir
config['Train']['dataset']['label_file_list'] = train_label_dir
config['Eval']['dataset']['data_dir'] = val_data_dir
config['Eval']['dataset']['label_file_list'] = val_label_dir
config['Eval']['loader']['batch_size_per_card'] = 10
# ...

- Có thể thay đổi Architecture (phù hợp với version model) để cải thiện performance của model

In [21]:
modified_config_str = ordered_dump(config, Dumper=yaml.SafeDumper)
print(modified_config_str)

Global:
  debug: false
  use_gpu: true
  epoch_num: 10
  log_smooth_window: 20
  print_batch_step: 10
  save_model_dir: ./output/rec_ppocr_v3
  save_epoch_step: 3
  eval_batch_step:
  - 0
  - 200
  cal_metric_during_train: true
  pretrained_model: /content/drive/MyDrive/Paddle_OCR/en_PP-OCRv3_rec_train/best_accuracy
  checkpoints: null
  save_inference_dir: ./output/save_inference
  use_visualdl: false
  infer_img: doc/imgs_words/ch/word_1.jpg
  character_dict_path: ./dataset/en_dict.txt
  max_text_length: 25
  infer_mode: false
  use_space_char: true
  distributed: true
  save_res_path: ./output/rec/predicts_v3_en.txt
Optimizer:
  name: Adam
  beta1: 0.9
  beta2: 0.999
  lr:
    name: Cosine
    learning_rate: 0.001
    warmup_epoch: 5
  regularizer:
    name: L2
    factor: 3.0e-05
Architecture:
  model_type: rec
  algorithm: SVTR_LCNet
  Transform: null
  Backbone:
    name: MobileNetV1Enhance
    scale: 0.5
    last_conv_stride:
    - 1
    - 2
    last_pool_type: avg
    last_pool

In [22]:
with open(config_path, 'w') as file:
  file.write(modified_config_str)

- Một số lỗi liên quan đến config trong quá trình train :
  - Không tìm thấy ảnh : vì file label_list.txt đã có đường dẫn ảnh nên data_dir chỉ cần đường dẫn tới folder chứa label_list

# Training

- Để tiếp tục train từ lần train trước :
  - Chỉnh sửa đường dẫn của pretrained_model trong file config
- Một số tham số về Architacture của v4_config sẽ không tương thích với model rec_v3
- Model v4 không có file model .pdparams

In [24]:
!python '/content/drive/MyDrive/Paddle_OCR/PaddleOCR/tools/train.py' -c '/content/drive/MyDrive/Paddle_OCR/dataset/v3_config.yml'

[2024/08/08 05:35:47] ppocr INFO: Architecture : 
[2024/08/08 05:35:47] ppocr INFO:     Backbone : 
[2024/08/08 05:35:47] ppocr INFO:         last_conv_stride : [1, 2]
[2024/08/08 05:35:47] ppocr INFO:         last_pool_kernel_size : [2, 2]
[2024/08/08 05:35:47] ppocr INFO:         last_pool_type : avg
[2024/08/08 05:35:47] ppocr INFO:         name : MobileNetV1Enhance
[2024/08/08 05:35:47] ppocr INFO:         scale : 0.5
[2024/08/08 05:35:47] ppocr INFO:     Head : 
[2024/08/08 05:35:47] ppocr INFO:         head_list : 
[2024/08/08 05:35:47] ppocr INFO:             CTCHead : 
[2024/08/08 05:35:47] ppocr INFO:                 Head : 
[2024/08/08 05:35:47] ppocr INFO:                     fc_decay : 1e-05
[2024/08/08 05:35:47] ppocr INFO:                 Neck : 
[2024/08/08 05:35:47] ppocr INFO:                     depth : 2
[2024/08/08 05:35:47] ppocr INFO:                     dims : 64
[2024/08/08 05:35:47] ppocr INFO:                     hidden_dims : 120
[2024/08/08 05:35:47] ppocr I

# Inference

- Test

In [28]:
from paddleocr import PaddleOCR

# Đường dẫn đến thư mục chứa model .pdparams
model_dir = '/content/drive/MyDrive/Paddle_OCR/output/rec_ppocr_v3/best_accuracy'

# Khởi tạo PaddleOCR với model của bạn
ocr = PaddleOCR(use_angle_cls=True, lang='en', rec_model_dir=model_dir)
# ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Đường dẫn đến hình ảnh bạn muốn nhận dạng
imgs_folder = '/content/drive/MyDrive/Paddle_OCR/dataset/original_images'
imgs_paths = os.listdir(imgs_folder)

for img in imgs_paths:
  img_path = os.path.join(imgs_folder, img)
  # Thực hiện dự đoán
  result = ocr.ocr(img_path, cls=False)
  print(f"Actual label : {img}\nPredict label : {result}")
  # # In kết quả
  # for line in result:
  #     print(line)

[2024/08/08 05:39:17] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/content/drive/MyDrive/Paddle_OCR/output/rec_ppocr_v3/best_accuracy', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text

# Eval